In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import shared_functions_wlp_wls as shared
import scipy.integrate
from halotools.empirical_models import PrebuiltHodModelFactory

/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/halo_table_cache.py:12: UserWarning: Some of the functionality of the HaloTableCache classrequires h5py to be installed.
  warn("Some of the functionality of the HaloTableCache class"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/user_supplied_ptcl_catalog.py:13: UserWarning: Most of the functionality of the sim_manager sub-package requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Most of the functionality of the sim_manager "
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/download_manager.py:39: UserWarning: Some of the functionality of the DownloadManager requires h5py to be installed,
which can be accomplished either with pip or conda
  warn("Some of the functionality of the DownloadManager requires h5py to be installed,\n"
/usr/local/lib/python2.7/dist-packages/halotools/sim_manager/cached_halo_catalog.py:15: UserWarning: Most of the functionality of the

In [2]:
SURVEY = 'SDSS'
if (SURVEY=='SDSS'):
    import params as pa
elif (SURVEY=='LSST_DESI'):
    import params_LSST_DESI as pa
else:
    print "We don't have support for that survey yet; exiting."
    exit()

Goal: reproduce the results of the right-hand panel of Zu & Mandelbaum 2016 ($<N_{\rm sat}>$ vs $M_h$ for a variety fo stellar mass bins

In [ ]:
Ms_bin_edges = np.asarray([10.2, 10.6, 11., 11.2, 11.4, 12.]) # 10**x in Msol / h^2

lgMh = np.linspace(11.5, 15.5, 100)
Mh = 10**(lgMh)

Nsat_above_ms = [0]*len(Ms_bin_edges)
for mi in range(0,len(Ms_bin_edges)):
    Nsat_above_ms[mi] = shared.get_Nsat_Zu(Mh, 10**Ms_bin_edges[mi], 'tot', 'SDSS')
    
Nsat_ms_inbin = [0]*(len(Ms_bin_edges)-1)
for mi in range(0,len(Nsat_ms_inbin)):
    Nsat_ms_inbin[mi] = Nsat_above_ms[mi] - Nsat_above_ms[mi+1]

In [ ]:
plt.figure()
plt.semilogy(lgMh, Nsat_ms_inbin[0], label='Bin 1')
plt.hold(True)
plt.semilogy(lgMh, Nsat_ms_inbin[1], label='Bin 2')
plt.hold(True)
plt.semilogy(lgMh, Nsat_ms_inbin[2], label='Bin 3')
plt.hold(True)
plt.semilogy(lgMh, Nsat_ms_inbin[3], label='Bin 4')
plt.hold(True)
plt.semilogy(lgMh, Nsat_ms_inbin[4], label='Bin 5')
plt.ylim(0.01, 100)
plt.ylabel('$N_{\\rm sat}$')
plt.xlabel('$M_h / h$')
plt.legend()
plt.show()

These are two high compared to the figure. Also, some of them are overlapping. What's wrong here?
Let's try to reproduce the right hand side of Figure 8, the Stellar - Halo mass relation

In [ ]:
Msvec = np.logspace(np.log10(7*10**8), np.log10(6*10**11), 100)
#Mh_out = shared.fSHMR_inverse(Msvec, 'SDSS')

#plt.figure()
#plt.loglog(Mh_out, Msvec)
#plt.ylabel('$M_{*}$')
#plt.xlabel('$M_h$')
#plt.xlim(10**11, 3*10**15)
#plt.ylim(7*10**8, 6*10**11)
#plt.show()

This is wrong too. Okay, let's try explicitly typing out the relation as in the paper rather than calling the funciton.

It's because I'm not accounting for the scatter. There is actually a formula in Ying's paper which provides a fit to the correct thing that we CAN directly invert (I think), not accounting for scatter. Let's try this.

In [ ]:
Msvec = np.logspace(np.log10(2*10**8), np.log10(10**13), 100)
Mh_get = shared.Mh_atfixed_Ms(Msvec)
lgMh_fit = np.log10(Mh_get)

plt.figure()
plt.loglog(Msvec, 10**lgMh_fit)
plt.ylabel('$M_{h}$')
plt.xlabel('$M_*$')
plt.ylim(5*10**10, 3*10**15)
plt.xlim(2*10**8, 10**12)
plt.show()

Okay, but what we actually want is $M_*(M_h)$. We know that at a fixed halo mass, the star mass has a log-normal scatter, and we know this scatter. So we can get, for each $M_h$, a mean $lnM_*(Mh)$ (from the relation of equation 19) and a series of other $ln M_*$ values drawn from a lognormal distribution aroumd this. The average of these $ln(M_*)$ value is then the mean value to associate with M_h. Try this.

In [ ]:
# Define an extended stellar vector for integrating over
Msvec = np.logspace(np.log10(2*10**8), np.log10(10**13), 100)
lnMstar = np.log(Msvec)

# For each halo mass
lgMh = np.linspace(11.5, 14., 100)
Mh = 10**(lgMh)
fshmr = np.zeros(len(lgMh))
for mi in range(0,len(lgMh)):
    lnMstar_mean = np.log(shared.get_fSHMR(Mh[mi], 'SDSS'))
    siglnM = shared.get_sigMs(Mh[mi], 'SDSS')
    # Define the lognormal distribution of M*
    p = 1. / (np.sqrt(2 * np.pi) * siglnM) * np.exp(-(lnMstar - lnMstar_mean)**2 / (2. * siglnM**2))
    #norm = np.sum(p)
    #p_normed = p / norm
    # Now sample from this distribution a bunch of lnM* values
    #lnMslist= np.random.choice(np.log(Msvec), p=p_normed, size = 10000)
    # Now get their mean
    lnMs_avg = scipy.integrate.simps(lnMstar * p, lnMstar)
    #lnMs_avg = np.mean(lnMslist)
    fshmr[mi] = np.exp(lnMs_avg)
    

The issue I am having now is the following.

I know $f_{\rm SHMR}(M_h) = exp(<ln(M_*(M_h))>)$

I know that $M_*$ is lognormally distributed at fixed halo mass, and I know its scatter.

But, how do I get its mean? This seems like a circular argument. I am trying to integrate:
$\int p(M_*,M_h) ln(M_*) d ln(M_*)$ 
to get the mean value $<ln(M_*(M_h))>$. But to get $p(M_*, M_h)$ in the first place, I require this mean value. How do I get out of this circle?

In [ ]:
plt.figure()
plt.loglog(Mh, fshmr)
plt.ylabel('$M_{*}$')
plt.xlabel('$M_h$')
plt.xlim(10**11, 3*10**15)
plt.ylim(7*10**8, 6*10**11)
plt.show()